In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import  transforms
from torch.utils.data import DataLoader
import wandb
import os
from PIL import Image
from torch.utils.data import Dataset

In [11]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name' : 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {'values': [0.001, 0.0001]},
        'batch_size': {'values': [16, 32]},
        'num_epochs': {'values': [5, 10]},
        'image_size': {'values' : [224]},
        'epochs':{'values': [5,10]},
        'droput_rate': {'values': [0.2, 0.5]},
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='VGG-16')

Create sweep with ID: ipbh00ef
Sweep URL: https://wandb.ai/fangselection123-happiest-minds-technologies/VGG-16/sweeps/ipbh00ef


In [20]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.image_paths=[]
        self.labels = []
        self.class_to_idx ={}
        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = { class_name: i for i, class_name in enumerate(class_names)}
        for class_name in class_names:
            class_dir= os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(self.class_to_idx[class_name])
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label



In [21]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding="same")
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding="same")
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding="same")
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding="same")

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding="same")
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding="same")
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding="same")

        self.conv8 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding="same")
        self.conv9 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding="same")
        self.conv10 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding="same")

        self.conv11 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding="same")
        self.conv12 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding="same")
        self.conv13 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding="same")

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 5)
        self.dropout = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
    def forward(self, x):
        x= self.relu(self.conv1(x))
        # print(x.shape)
        x=self.relu(self.conv2(x))
        # print(x.shape)

        x=self.pool(x)
        # print(x.shape)

        x=self.relu(self.conv3(x))
        # print(x.shape)

        x=self.relu(self.conv4(x))
        # print(x.shape)

        x=self.pool(x)
        # print(x.shape)

        x=self.relu(self.conv5(x))
        # print(x.shape)

        x=self.relu(self.conv6(x))
        # print(x.shape)

        x=self.relu(self.conv7(x))
        # print(x.shape)

        x=self.pool(x)
        # print(x.shape)

        x=self.relu(self.conv8(x))
        # print(x.shape)
        
        x=self.relu(self.conv9(x))
        # print(x.shape)

        x=self.relu(self.conv10(x))
        # print(x.shape)

        x=self.pool(x)
        # print(x.shape)

        x=self.relu(self.conv11(x))
        # print(x.shape)

        x=self.relu(self.conv12(x))
        # print(x.shape)

        x=self.relu(self.conv13(x))
        # print(x.shape)
        x=self.pool(x)
        x=self.flatten(x)
        # print(x.shape)

        x=self.dropout(self.fc1(x))
        x=self.relu(x)
        x=self.dropout(self.fc2(x))
        x=self.fc3(x)
        return x





        

In [22]:
from torchsummary import summary

model = VGG16()
summary(model, input_size=(3, 224, 224))  # ad

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [23]:
# import torchvision.models as models
# model = models.vgg16(pretrained=True)
# model.classifier[6] = nn.Linear(4096, 5)
# print(model)

In [24]:
def train():
    with wandb.init() as run:
        config= wandb.config
        train_dir="Data/flowers/train"
        test_dir="Data/flowers/val"
        transform = transforms.Compose([
            transforms.Resize((config.image_size, config.image_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])



        train_dataset = CustomDataset(train_dir, transform=transform)
        test_dataset = CustomDataset(test_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)


        class SimpleModel(nn.Module):
            def __init__(self):
                super(SimpleModel, self).__init__()
                self.flatten = nn.Flatten()
                self.fc1 = nn.Linear(config.img_size * config.img_size * 3, config.hidden_nodes)
                self.bn1 = nn.BatchNorm1d(config.hidden_nodes)
                # Output layer for 5 classes
                self.relu = nn.ReLU()
                self.dropout = nn.Dropout(p=0.2)
                self.fc2 = nn.Linear(config.hidden_nodes, 5)

            def forward(self, x):
                x = self.flatten(x)
                x = self.fc1(x)
                x= self.bn1(x)
                x = self.relu(x)
                x = self.dropout(x)
                x = self.fc2(x)
                return x  # No softmax needed; use CrossEntropyLoss
        model = VGG16()
        # -----------------------------
        # Training Setup
        # -----------------------------
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=1e-5) # l2 regularization
        # shedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

        # -----------------------------
        # Training Loop
        # -----------------------------
        EPOCHS = 10
        for epoch in range(EPOCHS):
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            for i, (images, labels) in enumerate(train_loader):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                # L1 regularization (manual)
                # l1_lambda = 1e-5
                # l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
                # loss += l1_lambda * l1_loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # shedule.step()

                train_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                batch_correct = (preds == labels).sum().item()
                train_correct += batch_correct
                train_total += labels.size(0)
                # Print every 10 batches
                if(i + 1) % 10 == 0:
                    batch_acc = batch_correct / labels.size(0)
                    print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")

            train_accuracy = train_correct / train_total
            wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_accuracy})
            print(f"Epoch {epoch+1} Summary - Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

            # -----------------------------
            # # Evaluation (Optional)
            # # -----------------------------
        model.eval()
        test_correct = 0
        test_total = 0
        test_loss = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                test_correct += (preds == labels).sum().item()
                test_total += labels.size(0)
        test_accuracy = test_correct / test_total
        wandb.log({"test_loss": test_loss, "test_accuracy": test_accuracy})
        print(f"Test Accuracy: {test_correct / test_total:.4f}")


        




In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: my4o60gi with config:
wandb: 	batch_size: 16
wandb: 	droput_rate: 0.2
wandb: 	epochs: 10
wandb: 	image_size: 224
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 5


[Batch 10/251] Loss: 1.6069, Batch Acc: 0.3125
[Batch 20/251] Loss: 1.6129, Batch Acc: 0.0625
[Batch 30/251] Loss: 1.6101, Batch Acc: 0.1250
[Batch 40/251] Loss: 1.6046, Batch Acc: 0.3125
[Batch 50/251] Loss: 1.6000, Batch Acc: 0.3125
[Batch 60/251] Loss: 1.6071, Batch Acc: 0.2500
[Batch 70/251] Loss: 1.5996, Batch Acc: 0.2500
[Batch 80/251] Loss: 1.6097, Batch Acc: 0.0000
[Batch 90/251] Loss: 1.6111, Batch Acc: 0.1875
[Batch 100/251] Loss: 1.6199, Batch Acc: 0.1875
[Batch 110/251] Loss: 1.6099, Batch Acc: 0.1250
[Batch 120/251] Loss: 1.6121, Batch Acc: 0.1250
[Batch 130/251] Loss: 1.5835, Batch Acc: 0.2500
[Batch 140/251] Loss: 1.6075, Batch Acc: 0.1875
[Batch 150/251] Loss: 1.6114, Batch Acc: 0.3125
[Batch 160/251] Loss: 1.6126, Batch Acc: 0.3125
[Batch 170/251] Loss: 1.6170, Batch Acc: 0.1875
[Batch 180/251] Loss: 1.6107, Batch Acc: 0.1250
[Batch 190/251] Loss: 1.6216, Batch Acc: 0.1875
[Batch 200/251] Loss: 1.6211, Batch Acc: 0.1875
[Batch 210/251] Loss: 1.6080, Batch Acc: 0.1250
[